# GHCN - Daily in DataBricks 
1. Create schema "ghcn"
2. Upload `station_metadata.csv` through DBFS
    - New -> Data -> uploda file -> Create table with UI -> Preview table
    - Click 'First row is header'
3. Read GHCN-daily data from azure blob storage URL into pandas dataframe,\
improve date time format, and write it into `ghcn_{year}`table
    - Modify `start_year` and `end_year` to fetch all yearly data
4. In SQL Queries, join GHCN and station_metadata and filter countries you are interested in
    - In this notebook: year 2022 in Central and South America 

In [ ]:
%sql
CREATE SCHEMA IF NOT EXISTS ghcn;

In [ ]:
import pandas as pd

new_columns = ['ID', 'Time', 'Element', 'Value', 'M-Flag', 'Q-Flag', 'S-Flag', 'OBS-Time']

start_year = 2022
end_year = 2022

for year in range(start_year, end_year + 1):
    
    URL = f'https://ghcn.blob.core.windows.net/ghcn/csv/daily/by_year/{year}.csv'

    df = pd.read_csv(URL, names = new_columns)
    
    df['Time'] = pd.to_datetime(df.Time, format='%Y%m%d')
    
    spark.createDataFrame(df).write.mode("overwrite").saveAsTable(f"ghcn.ghcn_{year}")


In [ ]:
%sql

SELECT *

FROM(
    SELECT *
    FROM ghcn.ghcn_2022, default.station_metadata_csv
    WHERE ghcn_2022.ID = station_metadata_csv.ID
    ) as join_dataset

WHERE join_dataset.Country IN ("Belize", "Costa Rica", "El Salvador", "Guatemala", "Honduras", "Mexico", "Nicaragua", "Panama", "Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador", "French Guiana", "Guyana", "Paraguay", "Peru", "Suriname", "Uruguay", "Venezuela")